In [ ]:
#BOOST GOOGLE COLAB MEMORY
#i = []
#while(True):
 #   i.append('a')

In [ ]:
#!nvidia-smi
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7ecca9c3-a177-3d51-addd-302efa42c11b)


IMPORTS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install git+https://github.com/openai/jukebox.git
#!pip install jukebox

  Cloning https://github.com/openai/jukebox.git to /tmp/pip-req-build-5i9eq9ok
  Running command git clone -q https://github.com/openai/jukebox.git /tmp/pip-req-build-5i9eq9ok
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 245kB 46.7MB/s 
     |████████████████████████████████| 1.6MB 60.4MB/s 
     |████████████████████████████████| 1.4MB 42.4MB/s 
  Created wheel for jukebox: filename=jukebox-1.0-cp36-none-any.whl size=197907 sha256=17728cf470d766ceb0e5449eaf366bffd2f514ec06e72867a576ca9cacb97300
  Stored in directory: /tmp/pip-ephem-wheel-cache-_mmporh7/wheels/bd/b6/f9/ad38a67dd989a522bbe6677e95efbc4607cdcf71e7249485fe
  Created wheel for fire: filename=fire-0.1.3-py2.py3-none-any.whl size=49706 sha256=9ef18fb0082c1df5bfd21dfce8680ccbd63e07c217946e974351dfdc6a01fad9
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Created wheel for librosa: filename=librosa-0.7.2-cp36-none-any.w

In [ ]:
import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

Using cuda True


MODEL INIT

In [ ]:
model = '1b_lyrics' # or '5b' or '1b_lyrics'
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = 3 if model in ('5b', '5b_lyrics') else 8
# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
hps.name = '/content/drive/MyDrive/MUSIC/JUKEBOX'
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
max_batch_size = 3 if model in ('5b', '5b_lyrics') else 16
hps.levels = 3
hps.hop_fraction = [.5,.5,.125]

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

Running  wget -O /root/.cache/jukebox/models/5b/vqvae.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar
Restored from /root/.cache/jukebox/models/5b/vqvae.pth.tar
0: Loading vqvae in eval mode
Creating cond. autoregress with prior bins [79, 2048], 
dims [384, 6144], 
shift [ 0 79]
input shape 6528
input bins 2127
Self copy is False
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v3_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v3_genre_ids.txt
Level:2, Cond downsample:None, Raw to tokens:128, Sample length:786432
Running  wget -O /root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar https://openaipublic.azureedge.net/jukebox/models/1b_lyrics/prior_level_2.pth.tar
Restored from /root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar
0: Loading prior in eval mode


In [ ]:
# Prime song creation using an arbitrary audio sample.
mode = 'primed'
codes_file=None
# Specify an audio file here.
audio_file = '/content/drive/MyDrive/MUSIC/Solo/NEW_MUSIC_SHOWCASE/WWW.wav'
# Specify how many seconds of audio to prime on.
prompt_length_in_seconds=12

CHECKPOINT

In [ ]:
#CHECKPOINT FOR MODEL ONLY
#if os.path.exists(hps.name):
  # Identify the lowest level generated and continue from there.
  #for level in [1, 2]:
   # data = f"{hps.name}/level_{level}/data.pth.tar"
   # if os.path.isfile(data):
    #  mode = 'upsample'
    #  codes_file = data
    #  print('Upsampling from level '+str(level))
    #  break
#print('mode is now '+mode)

-----------

In [ ]:
sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

In [ ]:
#NA?
sample_length_in_seconds = 50

hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

In [ ]:
metas = [dict(artist = "Kanye West",
            genre = "Hip Hop",
            total_length = hps.sample_length,
            offset = 0,
            lyrics = """In the valley of design,
            Designer for all the highs
            Lows full of the pain
            The story of you and I
            Snakes want me close
            Venom dripping like water
            I was born in the jungle
            Move froggy and you croak
            """,
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

In [ ]:
sampling_temperature = .98

lower_batch_size = 16
max_batch_size = 3 if model in ('5b', '5b_lyrics') else 16
lower_level_chunk_size = 32
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=0.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

MODEL

In [ ]:
if sample_hps.mode == 'ancestral':
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(len(priors))]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'upsample':
  assert sample_hps.codes_file is not None
  # Load codes.
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  del data
  print('Falling through to the upsample step later in the notebook.')
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
else:
  raise ValueError(f'Unknown sample mode {sample_hps.mode}.')

Sampling level 2
Sampling 6144 tokens for [0,6144]. Conditioning on 4134 tokens
Primed sampling 8 samples with temp=0.98, top_k=0, top_p=0.0
142/142 [00:22<00:00,  6.27it/s]
2010/2010 [03:00<00:00, 11.14it/s]
Sampling 6144 tokens for [768,6912]. Conditioning on 5376 tokens
Primed sampling 8 samples with temp=0.98, top_k=0, top_p=0.0
180/180 [00:29<00:00,  6.12it/s]
768/768 [01:08<00:00, 11.24it/s]
Sampling 6144 tokens for [1536,7680]. Conditioning on 5376 tokens
Primed sampling 8 samples with temp=0.98, top_k=0, top_p=0.0
180/180 [00:29<00:00,  6.01it/s]
768/768 [01:09<00:00, 11.04it/s]
Sampling 6144 tokens for [2304,8448]. Conditioning on 5376 tokens
Primed sampling 8 samples with temp=0.98, top_k=0, top_p=0.0
180/180 [00:29<00:00,  6.07it/s]
768/768 [01:08<00:00, 11.25it/s]
Sampling 6144 tokens for [3072,9216]. Conditioning on 5376 tokens
Primed sampling 8 samples with temp=0.98, top_k=0, top_p=0.0
180/180 [00:29<00:00,  6.07it/s]
768/768 [01:09<00:00, 11.12it/s]
Sampling 6144 tokens

UPSAMPLE

In [ ]:
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

Conditioning on 1 above level(s)
Checkpointing convs
Checkpointing convs
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v2_genre_ids.txt
Level:0, Cond downsample:4, Raw to tokens:8, Sample length:65536
Running  wget -O /root/.cache/jukebox/models/5b/prior_level_0.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar
Restored from /root/.cache/jukebox/models/5b/prior_level_0.pth.tar
0: Loading prior in eval mode
Conditioning on 1 above level(s)
Checkpointing convs
Checkpointing convs
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v2_artist_ids.txt
Loading artist IDs from /usr/local/lib/python3.6/dist-packages/jukebox/data/ids/v2_genre_ids.txt
Level:1, Cond downsample:4, Raw to tokens:32, Sample length:262144
Running  wget -O /root/.cache/jukebox/models/5b/prior_level_1.pth.tar https://openaipublic

In [ ]:
zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)